In [2]:
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import os
import photutils
import math

In [4]:
from Tutorial_functions import *
from mag_zero_est import *
from JG_Streaktools import *
from interface_setup import *
from plate_solve import *
from effective_psf import *
from beacon import *
from haversine import haversine, Unit

**-Distance 1-  62.3 meters**

Telescope
lat: 45.452961
lon: -73.794476
alt: 30m
accuracy: 4m

Source
lat: 45.452401
lon: -73.794680
alt: 30m
accuracy: 4m

In [210]:
dist = 62.3
theta = 0

exp_time = 1
delta_t = 10.0
lumens = 38
lam_nm = 600 #red
S_lambda0 = 1

gain_e_per_ADU = .9 

In [212]:
def V_lambda(lam_nm):
    """
    Photopic luminosity function approximation.
    Input: wavelength in nm
    Output: V from 0 to 1
    """
    lam = lam_nm
    
    # Simple Gaussian-like approximation centered at 555 nm
    return np.exp(-0.5*((lam - 555)/60)**2)

In [214]:
def lumens_to_radiant_intensity(lumens, lam_nm, beam_solid_angle=None):
    """
    Convert luminous flux (lumens) to radiant intensity (W/sr).

    Args:
        lumens: luminous flux [lm]
        lam_nm: wavelength of LED [nm]
        beam_solid_angle: optional solid angle in steradians.
                          If None, assumes isotropic.

    Returns:
        radiant_intensity [W/sr]
    """
    # Get V(lambda)
    V = V_lambda(lam_nm)

    if V == 0:
        raise ValueError("Luminosity function V(lambda) is zero at this wavelength.")

    # Radiant flux (W) from lumens
    radiant_flux = lumens / (683 * V)

    # Solid angle
    if beam_solid_angle is None:
        beam_solid_angle = 4 * np.pi  # isotropic

    # Radiant intensity (W/sr)
    I_rad = radiant_flux / beam_solid_angle

    return I_rad

In [216]:
def get_Fmeas(peak_rad, theta, dist, delta_t, exp_time, S_lambda0, err=True):
    """
    Computes the effective measured flux F_meas from a pulsed, 
    directional light source (integrating-sphere / LED), following:

        F_meas = (I_rad * cos(theta) / d^2) * S(lambda0) * (delta_t / exp_time)

    Args:
        peak_rad: radiant intensity of source [W/sr] or tuple (val, err)
        theta: angle between source axis and line of sight [radians] or tuple
        dist: distance to source [meters] or tuple
        delta_t: pulse width [seconds] or tuple
        exp_time: exposure time of image [seconds] or tuple
        S_lambda0: instrument throughput at wavelength lambda 0 (0–1), default = 1
        err: if True, propagate uncertainty (if tuples are given)

    Returns:
        F_meas (value or (value, uncertainty))
    """

    def unpack(x):
        if isinstance(x, tuple):
            return x[0], x[1]
        return float(x), 0.0

    I_rad, dI = unpack(peak_rad)
    th, dth = unpack(theta)
    d, dd = unpack(dist)
    Dt, dDt = unpack(delta_t)
    T, dT = unpack(exp_time)

    # Compute F_meas value
    F = (I_rad * np.cos(th) / d**2) * S_lambda0 * (Dt / T)

    if not err:
        return F


    term_I = (dI / I_rad)**2 if I_rad != 0 else 0
    term_th = (np.tan(th) * dth)**2
    term_d = (2 * dd / d)**2 if d != 0 else 0
    term_Dt = (dDt / Dt)**2 if Dt != 0 else 0
    term_T = (dT / T)**2 if T != 0 else 0

    dF = abs(F) * np.sqrt(term_I + term_th + term_d + term_Dt + term_T)

    return F, dF

In [218]:
def Fmeas_to_ADU(Fmeas, lam_nm, exp_time, S_lambda0, gain_e_per_ADU):
    """
    Convert irradiance Fmeas [W/m^2] to ADU counts including passband throughput.
    
    S_lambda0 = total system throughput at lam_nm (filter * optics * QE)
    """
    h = 6.62607015e-34
    c = 2.99792458e8

    lam = lam_nm * 1e-9
    A_pix = (2.9e-6)**2  # ASI585 pixel area

    # optical power on a pixel
    P_pix = Fmeas * A_pix

    # photon energy
    E_photon = h * c / lam

    # photons hitting pixel
    N_ph = (P_pix * exp_time * S_lambda0) / E_photon

    # electrons created (QE already included in S)
    N_e = N_ph


    ADU = N_e / gain_e_per_ADU
    return ADU

In [220]:
peak_rad = lumens_to_radiant_intensity(lumens, lam_nm, beam_solid_angle=1)

In [222]:
Fmeas = get_Fmeas(peak_rad, theta, dist, delta_t, exp_time, S_lambda0, err=False)

In [224]:
Fmeas_to_ADU(Fmeas, lam_nm, exp_time, S_lambda0, gain_e_per_ADU)

5359.9424544451722

**-Distance 2- 170.1 meters**

Telescope
lat: 45.452961
lon: -73.794476
alt: 30m
accuracy: 4m

Source
lat: 45.451428
lon: -73.794227
alt: 29m
accuracy: 5m

In [203]:
dist = 170.1

In [205]:
Fmeas = get_Fmeas(peak_rad, theta, dist, delta_t, exp_time, S_lambda0, err=False)

In [207]:
Fmeas_to_ADU(Fmeas, lam_nm, exp_time, S_lambda0, gain_e_per_ADU)

2647.6764654107074

**-Distance 3- 219 m**

Telescope
lat: 45.452717
lon: -73.795260
alt: 30m
accuracy: 5m

Source
lat: 45.450758
lon: -73.794449
alt: 27m
accuracy: 4m
x

In [107]:
dist = 219

In [109]:
Fmeas = get_Fmeas(peak_rad, theta, dist, delta_t, exp_time, S_lambda0, err=False)

In [111]:
Fmeas_to_ADU(Fmeas, lam_nm, exp_time, S_lambda0, gain_e_per_ADU)

782.67487521547605